# Setup
Use pip to download and install the necessary libraries if needed

In [ ]:
!pip install --upgrade pma_python
!pip install --upgrade pprint

In [ ]:
#
# helper libraries
#
import pprint as pp    # pretty print library is better to print list and dictionary structures
import os
import  matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pandas as pd

#
# pma_php library
#
from pma_python import core
print("pma_python library loaded; version", core.__version__)

# connection parameters to be used throughout this notebook
pma_start_slide_dir = "C:/wsi"

if core.is_lite():
    print ("Connected to PMA.start")
else:
    raise Exception("Unable to detect PMA.start! Please start PMA.start, or download it from http://free.pathomation.com")

In [ ]:
# only needed when debugging code for extra error messages:
core.set_debug_flag(True)

# PMA.start examples
example 10: identifying PMA.start

In [ ]:
# are you running PMA.start on localhost?
print("Are you running PMA.start? " + str(core.is_lite()))

# testing actual "full" PMA.core instance that's out there
pma_core_location = "https://host.pathomation.com/sandbox/2/pma.core"
print("Are you running PMA.start at " + pma_core_location + "? " + str(core.is_lite(pma_core_location)))

# testing against a non-existing end-point
pma_core_location = "https://www.google.com"
print("Are you running PMA.start at " + pma_core_location + "? " + str(core.is_lite(pma_core_location)))

example 20: getting version information about PMA.start

In [ ]:
# assuming we have PMA.start running; what's the version number?
print("You are running PMA.start version " + core.get_version_info())

example 30: connect to PMA.start

In [ ]:
sessionID = core.connect()        # no parameters needed for PMA.start

if (sessionID == None):
	print("Unable to connect to PMA.start");
else:
	print("Successfully connected to PMA.start; sessionID = ", sessionID)

example 40: getting drive letters from PMA.start

In [ ]:
print("You have the following drives in your system: ")
rootdirs = core.get_root_directories()
pp.pprint(rootdirs)

example 60: getting directories PMA.start

In [ ]:
rootdirs = core.get_root_directories();
print("Directories found in ", rootdirs[0],":")

dirs = core.get_directories(rootdirs[0])
pp.pprint(dirs)

example 70: get first non empty directory PMA.start

In [ ]:
slide_dir = core.get_first_non_empty_directory()
print (slide_dir)

example 80: getting slides PMA.start (recursively and non-recursively)

In [ ]:
slide_dir = pma_start_slide_dir
print("Looking for slides in " + slide_dir)
print()

print ("**Non-recursive:")
print(core.get_slides(slide_dir))

print ("\n**One-level deep recursion:")
print(core.get_slides(slide_dir, recursive = 1))

print ("\n**Full recursion:")
print(core.get_slides(slide_dir, recursive = True))


example 90: get UID for a slide in PMA.start

In [ ]:
slide_dir = pma_start_slide_dir

print("Looking for slides in " + slide_dir)
print()

for slide in core.get_slides(slide_dir):
	print (slide," - ", core.get_uid(slide))

example 100: get fingerprint for a slide in PMA.start

In [ ]:
slide_dir = pma_start_slide_dir

print("Looking for slides in " + slide_dir)
print()

for slide in core.get_slides(slide_dir):
	print (slide," - ", core.get_fingerprint(slide))

example 110: SlideInfo PMA.start

In [ ]:
slide_dir = pma_start_slide_dir
print("Looking for slides in", slide_dir)
print()

for slide in core.get_slides(slide_dir):
    print("***", slide)
    try:
        pp.pprint(core.get_slide_info(slide))
    except:
        print("**Unable to get slide info from this one")

example 115: SlideInfo on an empty file PMA.start

In [ ]:
slide_path = (os.getcwd() + "\\").replace("\\\\", "\\") + "temp.ndpi"
open(slide_path, 'a').close()
print("Created", slide_path)
translated_path = slide_path.replace("\\", "/")
print("Getting info on", translated_path)
core.get_slide_info(translated_path)
os.remove(slide_path)

example 120: slide dimensions PMA.start

In [ ]:
for slide in core.get_slides(pma_start_slide_dir):
    print("[" + slide + "]")
    try:
        xdim_pix, ydim_pix = core.get_pixel_dimensions(slide)
        xdim_phys, ydim_phys = core.get_physical_dimensions(slide)

        print("Pixel dimensions of slide: ", end="")
        print(xdim_pix, "x", ydim_pix)

        print("Slide surface area represented by image: ", end="")
        print(str(xdim_phys) + "µm x " + str(ydim_phys) + "µm = ", end="")
        print(xdim_phys * ydim_phys / 1E6, " mm2")
        
    except:
        print("**Unable to parse", slide)

example 130: get all files that make up a particular slide

In [ ]:
for slide in core.get_slides(pma_start_slide_dir):
    print(slide);
    pp.pprint(core.get_files_for_slide(slide))

example 140: who are you in PMA.start

In [ ]:
core.who_am_i()

example 150: investigate zoomlevels PMA.start

In [ ]:
for slide in core.get_slides(pma_start_slide_dir):
    print("***", slide)
    print("  max zoomlevel:", core.get_max_zoomlevel(slide))
    print("  zoomlevel list:")
    pp.pprint(core.get_zoomlevels_list(slide))
    print("  zoomlevel dictionary:")
    pp.pprint(core.get_zoomlevels_dict(slide))

example 160: investigate magnification characteristics PMA.start

In [ ]:
slide_dir = pma_start_slide_dir

slide_infos = []     # create blank list (to be converted into a pandas DataFrame later)

for slide in core.get_slides(slide_dir):
	dict = {
		"slide": core.get_slide_file_name(slide),
		"approx_mag": core.get_magnification(slide, exact=False),
		"exact_mag": core.get_magnification(slide, exact=True),
		"is_fluo": core.is_fluorescent(slide),
		"is_zstack": core.is_z_stack(slide)
		}
	slide_infos.append(dict)
	
df_slides = pd.DataFrame(slide_infos, columns=["slide","approx_mag","exact_mag", "is_fluo", "is_zstack"])
print(df_slides)    

example 170: get barcode from slide in PMA.start

In [ ]:
for slide in core.get_slides(pma_start_slide_dir, recursive=True):
    try:
        print(slide, " - ", core.get_barcode_text(slide))
    except:
        print("*Skipping", slide)

example 180: show a slide through PMA.start

In [ ]:
slides = core.get_slides(pma_start_slide_dir)
print(slides[0])
core.show_slide(slides[0])

example 190: slide label (URL) in PMA.start

In [ ]:
all_slides = core.get_slides(pma_start_slide_dir)
for sl in all_slides:
    print(core.get_label_url(sl))
plt.subplot(1, 2, 1)
plt.imshow(core.get_label_image(all_slides[0]))
plt.subplot(1, 2, 2)
plt.imshow(core.get_label_image(all_slides[1]))

example 200: slide label (URL) in PMA.start (using barcode alias methods)

In [ ]:
all_slides = core.get_slides(pma_start_slide_dir)
for sl in all_slides:
    print(core.get_barcode_url(sl))
plt.subplot(1, 2, 1)
plt.imshow(core.get_barcode_image(all_slides[0]))
plt.subplot(1, 2, 2)
plt.imshow(core.get_barcode_image(all_slides[1]))

example 210: thumbnail URL and image

In [ ]:
all_slides = core.get_slides(pma_start_slide_dir)
for sl in all_slides:
    print(core.get_thumbnail_url(sl))
plt.subplot(1, 2, 1)
plt.imshow(core.get_thumbnail_image(all_slides[0]))
plt.subplot(1, 2, 2)
plt.imshow(core.get_thumbnail_image(all_slides[1]))

Example 220: retrieving individual tiles in PMA.start

In [ ]:
slides = core.get_slides(pma_start_slide_dir)
slide = slides[0]
for zl in range(0, core.get_max_zoomlevel(slide)):
    (x, y, tot) = core.get_number_of_tiles(slide, zl)
    if tot > 16 and x >= 4 and y >= 4:
        break
for i in range(1,17):
    plt.subplot(4, 4, i)
    xr = 1 + (i-1) % 4
    yr = int((i-1) / 4) + 1
    tile = core.get_tile(slide, xr, yr, zl)
    plt.imshow(tile)

example 230: searching for slides in PMA.start

In [ ]:
slides = core.search_slides(pma_start_slide_dir, "mrxs")
pp.pprint(slides)

example 240: search for folders in PMA.start

In [ ]:
slides = core.search_slides(pma_start_slide_dir, "bladder")
pp.pprint(slides)